# Phase 3: Clustering & Segmentation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.manifold import TSNE
import plotly.express as px

In [2]:
# Load embeddings from Phase 2
embeddings = np.load('customer_embeddings.npy')  # Shape: (n_samples, 10)

## Determine Optimal Clusters  

In [3]:
# Evaluate cluster range (3-6)
cluster_range = range(3, 7)
silhouette_scores = []
db_scores = []

for n_clusters in cluster_range:
    # K-Means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(embeddings)
    
    # Metrics
    silhouette_scores.append(silhouette_score(embeddings, labels))
    db_scores.append(davies_bouldin_score(embeddings, labels))

# Plot metrics
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(cluster_range, silhouette_scores, 'bo-')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.title('Optimal Cluster Count')

plt.subplot(1, 2, 2)
plt.plot(cluster_range, db_scores, 'go-')
plt.xlabel('Number of clusters')
plt.ylabel('Davies-Bouldin Score')
plt.title('Lower is Better')
plt.savefig('cluster_metrics.png', bbox_inches='tight')
plt.close()

# Select best cluster count (max silhouette)
optimal_clusters = cluster_range[np.argmax(silhouette_scores)]
print(f"Optimal clusters: {optimal_clusters} (Silhouette: {max(silhouette_scores):.3f})")

Optimal clusters: 5 (Silhouette: 0.243)


## Final Clustering  


In [4]:
# K-Means with optimal clusters
final_kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
cluster_labels = final_kmeans.fit_predict(embeddings)

# Alternative: Gaussian Mixture Model
# gmm = GaussianMixture(n_components=optimal_clusters, random_state=42)
# cluster_labels = gmm.fit_predict(embeddings)

## Cluster Analysis 

In [5]:
# Load original data with features
df = pd.read_csv('preprocessed_campaign_data.csv')
df['Cluster'] = cluster_labels

# Profile clusters
cluster_profiles = df.groupby('Cluster').agg({
    'Income': 'median',
    'Total_Spend': 'median',
    'Recency': 'median',
    'Family_Size': 'median',
    'MntWines': 'sum',
    'NumWebVisitsMonth': 'mean'
}).reset_index()

# Visualize profiles
plt.figure(figsize=(12, 8))
sns.heatmap(cluster_profiles.set_index('Cluster'), 
            annot=True, cmap='Blues', fmt='.1f')
plt.title('Customer Segment Profiles', pad=20)
plt.savefig('cluster_profiles.png', bbox_inches='tight')
plt.close()

## Interactive Visualization 

In [6]:
# t-SNE for 2D visualization
tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(embeddings)

# Create interactive plot
plot_df = pd.DataFrame({
    'x': embeddings_2d[:, 0],
    'y': embeddings_2d[:, 1],
    'Cluster': cluster_labels,
    'Income': df['Income'],
    'Total_Spend': df['Total_Spend']
})

fig = px.scatter(plot_df, x='x', y='y', color='Cluster',
                 hover_data=['Income', 'Total_Spend'],
                 title='Customer Segments (t-SNE)')
fig.write_html('interactive_clusters.html')

## Save Results

In [7]:
# Save cluster labels
df.to_csv('clustered_customers.csv', index=False)

# Generate cluster descriptions
cluster_descriptions = []
for cluster in range(optimal_clusters):
    subset = df[df['Cluster'] == cluster]
    desc = f"""
    Cluster {cluster} (n={len(subset)}):
    - Median Income: ${subset['Income'].median():,.0f}
    - Median Spend: ${subset['Total_Spend'].median():,.0f}
    - Key Characteristics: {'High' if subset['MntWines'].median() > 300 else 'Low'} wine spenders,
      {'Frequent' if subset['Recency'].median() < 30 else 'Infrequent'} purchasers
    """
    cluster_descriptions.append(desc)

with open('cluster_descriptions.txt', 'w') as f:
    f.write("\n".join(cluster_descriptions))

print("Clustering completed!")
print(f"- Silhouette Score: {silhouette_score(embeddings, cluster_labels):.3f}")
print("- Visualizations saved: cluster_metrics.png, cluster_profiles.png")
print("- Interactive plot: interactive_clusters.html")

Clustering completed!
- Silhouette Score: 0.243
- Visualizations saved: cluster_metrics.png, cluster_profiles.png
- Interactive plot: interactive_clusters.html
